In [1]:
!mkdir embeddings

In [2]:
!wget -P 'drive/My Drive/news_rec_2020/embeddings' http://files.deeppavlov.ai/deeppavlov_data/bert/sentence_ru_cased_L-12_H-768_A-12_pt.tar.gz
!tar -zxvf 'drive/My Drive/news_rec_2020/embeddings/sentence_ru_cased_L-12_H-768_A-12_pt.tar.gz'

--2020-04-02 11:18:12--  http://files.deeppavlov.ai/deeppavlov_data/bert/sentence_ru_cased_L-12_H-768_A-12_pt.tar.gz
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 93.175.29.74
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 661614603 (631M) [application/octet-stream]
Saving to: ‘drive/My Drive/news_rec_2020/embeddings/sentence_ru_cased_L-12_H-768_A-12_pt.tar.gz’

sentence_ru_cased_L 100%[===================>] 630.96M  8.43MB/s    in 94s     

2020-04-02 11:19:46 (6.73 MB/s) - ‘drive/My Drive/news_rec_2020/embeddings/sentence_ru_cased_L-12_H-768_A-12_pt.tar.gz’ saved [661614603/661614603]

sentence_ru_cased_L-12_H-768_A-12_pt/
sentence_ru_cased_L-12_H-768_A-12_pt/pytorch_model.bin
sentence_ru_cased_L-12_H-768_A-12_pt/bert_config.json
sentence_ru_cased_L-12_H-768_A-12_pt/vocab.txt


In [1]:
!pip3 install --user deeppavlov
!pip3 install --user transformers

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.4.0
True


In [2]:
# http://docs.deeppavlov.ai/en/master/features/models/bert.html#bert-as-embedder

from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs

bert_config = read_json(configs.embedder.bert_sentence_embedder)
bert_config['metadata']['variables']['BERT_PATH'] = 'sentence_ru_cased_L-12_H-768_A-12_pt'

bert = build_model(bert_config)

texts = ['Как твои дела?', 'Как твое здоровье?']
max_emb, mean_emb, pooler_output = bert(texts)
pooler_output

[nltk_data] Downloading package punkt to /home/vnkljukin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vnkljukin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/vnkljukin/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/vnkljukin/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


array([[-0.191129  , -0.47446856,  0.3506737 , ..., -0.2261294 ,
         0.46362826,  0.45683557],
       [-0.1404717 , -0.4969943 ,  0.18004802, ..., -0.10774759,
         0.24053936,  0.29534283]], dtype=float32)

In [3]:
texts = ['Как твои дела?', 'Как твое здоровье?', 'Как твои дела? Как твое здоровье?']
_, _, pooler_output = bert(texts)
pooler_output

array([[-0.19112898, -0.4744686 ,  0.35067368, ..., -0.22612953,
         0.4636283 ,  0.4568356 ],
       [-0.14047155, -0.4969943 ,  0.18004806, ..., -0.10774761,
         0.24053934,  0.29534283],
       [-0.12264521, -0.41991535,  0.221898  , ..., -0.10056899,
         0.24817692,  0.20215514]], dtype=float32)

In [4]:
!nvidia-smi

Thu Apr  2 12:40:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64       Driver Version: 440.64       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro RTX 4000     On   | 00000000:02:00.0 Off |                  N/A |
| 30%   27C    P8    14W / 125W |     12MiB /  7982MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 4000     On   | 00000000:05:00.0 Off |                  N/A |
| 30%   33C    P0    37W / 125W |   1529MiB /  7982MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  Q

In [5]:
def process_line(line):
    return json.loads(line)

In [6]:
items_count = 0
with open('items.json') as items_file:
    for line in items_file:
        items_count += 1
items_count

328050

In [7]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from collections import defaultdict
from multiprocessing import Pool, cpu_count

In [9]:
items = []
with open('items.json') as items_file:
    with Pool(28) as p:
        items = list(tqdm(p.imap(process_line, items_file), total=items_count))
items = pd.DataFrame(items).set_index('itemId')
items.head()

,content,image,title
itemId,,,
0,"Согласитесь, дорогие любители собак, до чего ж...","[-0.169, 0.129, 0.067, 0.019, 0.281, -0.245, 0...",Пять забавных «морщинистых» пород собак
1,"Контуры Третьей Поперечной улицы, состоявшей ...","[-0.158, -0.112, -0.325, 0.05, -0.114, 0.002, ...",История улицы Ирининской в Гомеле
2,Источник: http://infodays.ru Вообще он как-то ...,"[0.084, -0.181, 0.008, 0.34, -0.03, -0.197, -0...",Зачем Дудь всё время спрашивает гостей програм...
3,41-летняя Светлана Зейналова решила окрестить ...,"[0.034, -0.119, -0.062, 0.025, 0.128, -0.041, ...",Светлана Зейналова крестила младшую дочь
4,«Организованные преступные группировки ГБАО де...,"[-0.061, -0.015, -0.198, -0.047, 0.054, 0.029,...","ГКНБ: бандиты в ГБАО делают вид, что рассталис..."


In [10]:
items_title = items.title.values
items_title[:5]

array(['Пять забавных «морщинистых» пород собак',
       'История улицы Ирининской в  Гомеле',
       'Зачем Дудь всё время спрашивает гостей программы о сексе',
       'Светлана Зейналова крестила младшую дочь',
       'ГКНБ: бандиты в ГБАО делают вид, что расстались с прошлым'],
      dtype=object)

In [11]:
import gc
gc.collect()

12

In [12]:
bert_titles = []
batch_size = 512
total_len = items_title.shape[0]
UNK = total_len
total_batches = total_len // batch_size + 1

cur_batch_num = 0
for cur_batch_num in tqdm(range(total_batches)):
    cur_batch = items_title[cur_batch_num * batch_size : min((cur_batch_num + 1) * batch_size, total_len)]
    with torch.no_grad():
        bert_titles.append(bert(cur_batch)[2])
bert_titles.append(np.zeros_like(bert_titles[0][0, :])[np.newaxis, :])
len(bert_titles)

642

In [13]:
items_content = items.content.values
items_content[0]

'Согласитесь, дорогие любители собак, до чего же мило и обаятельно выглядят собаки с забавными складками кожи – морщинками! Некоторые такие породы в свое время были остромодными и, соответственно, дефицитными. Цена – не радовала… И все-таки, как полагают наши британские и испанские коллеги, складки-морщинки – знак собачьей нежности и красоты, не больше и не меньше. Давайте с ними согласимся и заодно посмотрим на подборку из пяти «морщинистых» пород.\n  Столь забавные складки-морщинки, как вы понимаете, появляются у собак не с возрастом и не от старости, как можно было бы подумать. В первую очередь, они возникают из-за генетической предрасположенности, обусловленной большим содержанием в организме так называемой гиалуроновой кислоты. Именно она «ответственна» за столь интересную, мясистую и в глубоких складках кожу. Кстати, мимические морщины у собак тоже бывают, прямо как и у людей, но сегодня речь не о них – а о тех очаровательных складках, которые сводят с ума поклонников данных поро

In [38]:
bert_content = []
batch_size = 128
total_len = items_content.shape[0]
UNK = total_len
total_batches = total_len // batch_size + 1

cur_batch_num = 0
for cur_batch_num in tqdm(range(total_batches)):
    cur_batch = items_content[cur_batch_num * batch_size : min((cur_batch_num + 1) * batch_size, total_len)]
    cur_batch = list(map(lambda x: ' '.join(x.split(' ')[:64]), cur_batch))
    with torch.no_grad():
        bert_content.append(bert(cur_batch)[2])
bert_content.append(np.zeros_like(bert_content[0][0, :])[np.newaxis, :])
len(bert_content)

RuntimeError: input sequence after bert tokenization cannot exceed 512 tokens.

In [25]:
gc.collect()
torch.cuda.empty_cache()

In [41]:
for cur_batch_num in tqdm(range(cur_batch_num, total_batches)):
    cur_batch = items_content[cur_batch_num * batch_size : min((cur_batch_num + 1) * batch_size, total_len)]
    cur_batch = list(map(lambda x: ' '.join(x.split(' ')[:64]), cur_batch))
    with torch.no_grad():
        try:
            bert_content.append(bert(cur_batch)[2])
        except RuntimeError:
            cur_batch = list(map(lambda x: ' '.join(x.split(' ')[:16]), cur_batch))
            bert_content.append(bert(cur_batch)[2])
bert_content.append(np.zeros_like(bert_content[0][0, :])[np.newaxis, :])

In [42]:
len(bert_content)

2564

In [44]:
titles_embeddings = np.concatenate(bert_titles, axis=0)
content_embeddings = np.concatenate(bert_content, axis=0)
titles_embeddings.shape, content_embeddings.shape

((328051, 768), (328051, 768))

In [46]:
items['title_embedding'] = pd.Series(map(lambda x:[x], titles_embeddings)).apply(lambda x:x[0])
items['content_embedding'] = pd.Series(map(lambda x:[x], content_embeddings)).apply(lambda x:x[0])
items.head()

,content,image,title,title_embedding,content_embedding
itemId,,,,,
0,"Согласитесь, дорогие любители собак, до чего ж...","[-0.169, 0.129, 0.067, 0.019, 0.281, -0.245, 0...",Пять забавных «морщинистых» пород собак,"[-0.13784322, -0.2494488, -0.052675568, 0.0674...","[0.0017429275, -0.13621575, -0.005446423, -0.0..."
1,"Контуры Третьей Поперечной улицы, состоявшей ...","[-0.158, -0.112, -0.325, 0.05, -0.114, 0.002, ...",История улицы Ирининской в Гомеле,"[-0.18086486, -0.438953, 0.20550759, -0.586738...","[-0.20222285, -0.3154842, 0.07183142, -0.21042..."
2,Источник: http://infodays.ru Вообще он как-то ...,"[0.084, -0.181, 0.008, 0.34, -0.03, -0.197, -0...",Зачем Дудь всё время спрашивает гостей програм...,"[-0.29486495, -0.086909056, 0.0169366, -0.2004...","[-0.13806573, -0.14691418, 0.0037532346, -0.24..."
3,41-летняя Светлана Зейналова решила окрестить ...,"[0.034, -0.119, -0.062, 0.025, 0.128, -0.041, ...",Светлана Зейналова крестила младшую дочь,"[-0.011012467, -0.5578447, 0.37506068, -0.5651...","[0.07046502, -0.40018868, 0.20598526, -0.52284..."
4,«Организованные преступные группировки ГБАО де...,"[-0.061, -0.015, -0.198, -0.047, 0.054, 0.029,...","ГКНБ: бандиты в ГБАО делают вид, что рассталис...","[-0.18723457, -0.45224637, 0.14010152, -0.6870...","[-0.15916014, -0.47234368, 0.06936865, -0.6120..."


In [54]:
np.save('/data/vnkljukin/title_embeddings_bert', titles_embeddings)

In [55]:
np.save('/data/vnkljukin/content_embeddings_bert', content_embeddings)

In [57]:
del bert
gc.collect()
torch.cuda.empty_cache()